In this notebook, we will calculate features using Trotterization with a noise-free classical simulator (AerSimulator).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['id', 'reset', 'sx', 'x', 'measure', 'cz']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2

### Prepare parameters and load dataset

In [7]:
n_qubits = 52

C = 3
n_samples = 55
n_shots = 100

if n_qubits <= 50:  # n_qubits <= 50
    threshold = 0.6

    # Load dataset (orient="records")
    df_n_step = pd.read_json(
        f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
        orient="records",
    )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 50
    threshold = None
    n_features = 20
    max_time = 20
    # t_k = k * pi / C
    times = np.arange(0, max_time, np.pi / C)

    # n_qubits > 50 の場合は、n_step は 1 から 7 の整数値を取る。
    # まず 1から7までの20個の等間隔の数値を生成
    n_step_array = np.linspace(1, 7, 20)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

n_qubits: 52
C: 3
threshold: None
n_samples: 55
n_shots: 100
n_features: 20
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306
 12.56637061 13.61356817 14.66076572 15.70796327 16.75516082 17.80235837
 18.84955592 19.89675347]
n_step_array: [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.021581075300000002, -0.004814892800000001, ...",0.874333,NaN,NaN
1,1,"[-0.0424831577, 0.024743239900000002, 0.014216...",1.264982,NaN,NaN
2,2,"[-0.019849499200000002, 0.039776682300000005, ...",1.043115,NaN,NaN
3,3,"[0.0038696408, -0.0280307067, 0.01233447190000...",0.878768,NaN,NaN
4,4,"[-0.038509265, -0.021418483500000002, -0.02916...",0.920255,NaN,NaN
5,5,"[0.0169817178, 0.021851908, -0.027246614500000...",1.089466,NaN,NaN
6,6,"[-0.0100976512, 0.0122639897, -0.0205218139, -...",0.917398,NaN,NaN
7,7,"[0.0228687258, -0.0005996629, 0.00761107880000...",0.887918,NaN,NaN
8,8,"[-0.024747151000000002, 0.0313821979, 0.010417...",1.136008,NaN,NaN
9,9,"[0.0145011702, 0.0076980293, -0.0377251604, 0....",1.025680,NaN,NaN


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    # "backend": backend_dm_noiseless,  # This is important!!
    "backend": backend_mps_noiseless,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [ ]:
%%time
# For simulator, we need job instances instead of job ids
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55


Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Preparing circuits for sample 28/55
P

In [10]:
jobs[0].result()

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=52>)), metadata={'shots': 100, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(m

In [11]:
%%time
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_15 Im,f_16 Re,f_16 Im,f_17 Re,f_17 Im,f_18 Re,f_18 Im,f_19 Re,f_19 Im,expected_value
0,0,1.0,0.990335,-0.140131,0.935088,-0.343236,0.876400,-0.481272,0.834926,-0.561248,...,-0.655305,-0.508890,-0.503519,-0.516433,-0.430461,-0.611795,-0.381323,-0.652169,-0.212780,0.874333
1,1,1.0,0.968916,0.254563,0.896513,0.448513,0.760799,0.642951,0.471514,0.870962,...,-0.430950,-0.585321,-0.456069,-0.403886,-0.606116,-0.243276,-0.626192,-0.133462,-0.600906,1.264982
2,2,1.0,1.012136,-0.066937,0.975801,0.120881,0.992056,0.146713,0.973774,0.117321,...,0.355610,0.612567,0.281535,0.725087,0.387620,0.597175,0.353104,0.516726,0.353121,1.043115
3,3,1.0,0.983090,-0.127018,0.943223,-0.310373,0.873060,-0.434357,0.853118,-0.579992,...,-0.703174,-0.417120,-0.518856,-0.548516,-0.569588,-0.471315,-0.454271,-0.608160,-0.274665,0.878768
4,4,1.0,0.990825,-0.044328,0.953954,-0.109870,0.935098,-0.295961,0.942826,-0.331028,...,-0.546651,0.059300,-0.428816,-0.105781,-0.444084,-0.065755,-0.238068,-0.043252,-0.400786,0.920255
5,5,1.0,0.984808,0.123096,0.942816,0.257679,0.916161,0.282754,0.871443,0.299144,...,0.555437,0.082168,0.538283,0.091573,0.523464,0.104027,0.565047,0.016128,0.518401,1.089466
6,6,1.0,0.976875,-0.175258,0.985709,-0.173143,0.980519,-0.220415,0.906702,-0.325409,...,-0.689290,-0.065230,-0.677529,-0.122581,-0.649749,-0.296856,-0.572605,-0.342930,-0.510783,0.917398
7,7,1.0,0.969847,-0.199740,0.894293,-0.321155,0.911047,-0.369449,0.928104,-0.403266,...,-0.510656,-0.230319,-0.398313,-0.438855,-0.354268,-0.362942,-0.258405,-0.281061,-0.230661,0.887918
8,8,1.0,0.998392,0.057557,0.944704,0.311182,0.940153,0.347292,0.820951,0.443328,...,0.531024,-0.141592,0.583397,-0.228972,0.411912,-0.227163,0.302815,-0.252520,0.325935,1.136008
9,9,1.0,1.000070,-0.016138,0.999738,0.037723,0.999469,0.051583,0.986483,0.235267,...,0.321076,0.783035,0.298925,0.801313,0.463570,0.753805,0.403333,0.761482,0.416947,1.025680
